<a href="https://colab.research.google.com/github/varshithr/ner/blob/main/NER_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pinecone-client pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pinecone

In [ ]:
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer


In [ ]:
api_key ='92195b0e-7166-493b-9391-7e9735fbf228'

In [ ]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=api_key)

index_name = "entity-index"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # dimension supported by mpnet model
        metric='cosine',  # can be 'euclidean' too
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
index = pc.Index(index_name)


In [ ]:
orgs_df = pd.read_csv("firm_names.csv")

In [ ]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
embedding_model = AutoModel.from_pretrained(embedding_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
import torch

In [ ]:
def generate_embedding(text):
    inputs = embedding_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad(): # gradient calculation notrequired for embedding models
        embeddings = embedding_model(**inputs).last_hidden_state.mean(dim=1) # Gen embeddings and flatten result into a single vector
    return embeddings.squeeze().numpy() # Convert vector in pt tensor output to numpy array

In [ ]:
def insert_into_pinecone(df, label):
    vectors = []
    batch_size = 100
    for _, row in df.iterrows():
        text = row['FULLNAME']
        embedding = generate_embedding(text)
        vectors.append((text, embedding, {'label': label}))

        if len(vectors) >= batch_size:
            index.upsert(vectors=vectors)
            print(f"Inserted {len(vectors)} vectors into Pinecone")
            vectors = []

    if vectors:
        index.upsert(vectors=vectors)

In [ ]:
insert_into_pinecone(orgs_df, 'ORG')

NameError: name 'orgs_df' is not defined

In [ ]:
persons_df = pd.read_csv("person_names.csv")

In [ ]:
insert_into_pinecone(persons_df, 'PER')

Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors into Pinecone
Inserted 100 vectors

In [ ]:
def bytes_to_mb(bytes_value):
  return bytes_value / (1024 * 1024)

bytes_value = 4194304
mb_value = bytes_to_mb(bytes_value)
print(mb_value)

4.0


In [ ]:
ner_model_name = "Babelscape/wikineural-multilingual-ner"
ner_tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [ ]:
def extract_entities(text):
    ner_results = ner_pipeline(text)
    entities = [result['word'] for result in ner_results if result['entity_group'] in ['PER', 'ORG']]
    return entities

In [ ]:
extract_entities("Morgan Stanley")

['Morgan Stanley']

In [ ]:
def query_pinecone(entities, top_k=5):
    results = []
    for entity in entities:
        embedding = generate_embedding(entity)
        query_result = index.query(embedding.tolist(), top_k=top_k)
        results.extend(query_result['matches'])
    return results

In [ ]:
def query_pinecone(entities, top_k=5):  # Assuming top_k is 5 if not provided
    results = []
    for entity in entities:
        embedding = generate_embedding(entity)
        # Use keyword arguments for query()
        query_result = index.query(vector=embedding.tolist(), top_k=top_k)
        results.extend(query_result['matches'])
    return results

In [ ]:
def combine_input_with_retrieved_docs(text, retrieved_docs):
    combined_text = text + "\n\nRetrieved Documents:\n"
    for doc in retrieved_docs:
        # Access the 'metadata' from the 'data' attribute of the ScoredVector object
        combined_text += doc.data.get('metadata', {}).get('text', '') + "\n"
    return combined_text

In [ ]:
# Load a generative model (e.g., GPT-2)
gen_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gen_model = GPT2LMHeadModel.from_pretrained("gpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def generate_output(combined_input):
    inputs = gen_tokenizer.encode(combined_input, return_tensors="pt")
    outputs = gen_model.generate(inputs, max_length=500, num_return_sequences=1)
    generated_text = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [ ]:
def main(text):
    entities = extract_entities(text)
    retrieved_docs = query_pinecone(entities)
    combined_input = combine_input_with_retrieved_docs(text, retrieved_docs)
    final_output = generate_output(combined_input)
    print("Final Output:", final_output)


In [ ]:

text = "Wells Fargo"
main(text)

AttributeError: 'NoneType' object has no attribute 'get'